In [27]:
import pandas as pd
import sqlite3

# Load CSV
df = pd.read_csv('/cleaned_superstore.csv')

# Create SQLite DB
conn = sqlite3.connect('superstore.db')

# Save data into SQL table
df.to_sql('Superstore', conn, if_exists='replace', index=False)


9994

In [28]:
import pandas as pd

query = """
SELECT Category, SUM(Sales) AS TotalSales
FROM Superstore
GROUP BY Category
"""

pd.read_sql_query(query, conn)


,Category,TotalSales
0,Furniture,741999.7953
1,Office Supplies,719047.0320
2,Technology,836154.0330


In [29]:
pd.read_sql_query("""
SELECT
    SUM(Sales)   AS Total_Sales,
    SUM(Profit)  AS Total_Profit,
    SUM(Quantity) AS Total_Quantity
FROM Superstore;
""", conn)

,Total_Sales,Total_Profit,Total_Quantity
0,2.297201e+06,286397.0217,37873


In [30]:
pd.read_sql_query("""
SELECT
    SUBSTR("Order Date", 1, 7) AS Month,   -- YYYY-MM
    SUM(Sales) AS Monthly_Sales
FROM Superstore
GROUP BY SUBSTR("Order Date", 1, 7)
ORDER BY Month;
""",conn)

,Month,Monthly_Sales
0,01-01-2,1481.8280
1,01-02-2,630.8700
2,01-03-2,10131.1314
3,01-04-2,8254.3560
4,01-05-2,5077.1760
...,...,...
361,31-05-2,6390.6970
362,31-07-2,7444.3480
363,31-08-2,5881.1150
364,31-10-2,12847.4620


In [31]:
pd.read_sql_query("""
WITH monthly AS (
    SELECT
        SUBSTR("Order Date", 1, 4) AS Year,
        SUBSTR("Order Date", 6, 2) AS Month,
        SUM(Sales) AS Monthly_Sales
    FROM Superstore
    GROUP BY Year, Month
),
yoy AS (
    SELECT
        m1.Year,
        m1.Month,
        m1.Monthly_Sales AS Current_Year_Sales,
        m2.Monthly_Sales AS Last_Year_Sales,
        ROUND(
            ((m1.Monthly_Sales - m2.Monthly_Sales) / m2.Monthly_Sales) * 100,
            2
        ) AS YoY_Percentage
    FROM monthly m1
    LEFT JOIN monthly m2
        ON m1.Month = m2.Month
        AND m1.Year = m2.Year + 1
)
SELECT *
FROM yoy
ORDER BY Year, Month;
""",conn)

,Year,Month,Current_Year_Sales,Last_Year_Sales,YoY_Percentage
0,01-0,-2,53936.9554,None,None
1,01-1,-2,41587.6110,None,None
2,02-0,-2,49043.8070,None,None
3,02-1,-2,56095.3170,None,None
4,03-0,-2,43919.9226,None,None
...,...,...,...,...,...
57,29-1,-2,13709.5280,None,None
58,30-0,-2,42006.9654,None,None
59,30-1,-2,22851.3060,None,None
60,31-0,-2,29489.6808,None,None


In [32]:
pd.read_sql_query("""
SELECT
    "Product Name",
    SUM(Sales) AS Total_Sales
FROM Superstore
GROUP BY "Product Name"
ORDER BY Total_Sales DESC
LIMIT 10;
""",conn)

,Product Name,Total_Sales
0,Canon imageCLASS 2200 Advanced Copier,61599.824
1,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.384
2,Cisco TelePresence System EX90 Videoconferenci...,22638.480
3,HON 5400 Series Task Chairs for Big and Tall,21870.576
4,GBC DocuBind TL300 Electric Binding System,19823.479
5,GBC Ibimaster 500 Manual ProClick Binding System,19024.500
6,Hewlett Packard LaserJet 3310 Copier,18839.686
7,HP Designjet T520 Inkjet Large Format Printer ...,18374.895
8,GBC DocuBind P400 Electric Binding System,17965.068
9,High Speed Automatic Electric Letter Opener,17030.312


In [33]:
pd.read_sql_query("""
SELECT
    "Customer Name",
    SUM(Sales) AS Total_Revenue
FROM Superstore
GROUP BY "Customer Name"
ORDER BY Total_Revenue DESC
LIMIT 10;
""",conn)

,Customer Name,Total_Revenue
0,Sean Miller,25043.050
1,Tamara Chand,19052.218
2,Raymond Buch,15117.339
3,Tom Ashbrook,14595.620
4,Adrian Barton,14473.571
5,Ken Lonsdale,14175.229
6,Sanjit Chand,14142.334
7,Hunter Lopez,12873.298
8,Sanjit Engle,12209.438
9,Christopher Conant,12129.072


In [34]:
pd.read_sql_query("""
SELECT
    Category,
    SUM(Sales) AS Total_Sales,
    SUM(Profit) AS Total_Profit,
    ROUND((SUM(Profit) * 1.0 / SUM(Sales)) * 100, 2) AS Profit_Margin_Percentage
FROM Superstore
GROUP BY Category
ORDER BY Profit_Margin_Percentage DESC;
""",conn)


,Category,Total_Sales,Total_Profit,Profit_Margin_Percentage
0,Technology,836154.0330,145454.9481,17.40
1,Office Supplies,719047.0320,122490.8008,17.04
2,Furniture,741999.7953,18451.2728,2.49


In [35]:
pd.read_sql_query("""
SELECT
    Region,
    SUM(Sales) AS Total_Sales,
    SUM(Profit) AS Total_Profit
FROM Superstore
GROUP BY Region
ORDER BY Total_Sales DESC;
""",conn)


,Region,Total_Sales,Total_Profit
0,West,725457.8245,108418.4489
1,East,678781.2400,91522.7800
2,Central,501239.8908,39706.3625
3,South,391721.9050,46749.4303


In [36]:
pd.read_sql_query("""
SELECT
    Discount,
    COUNT(*) AS Number_of_Orders,
    SUM(Sales) AS Total_Sales,
    SUM(Profit) AS Total_Profit,
    ROUND((SUM(Profit) * 1.0 / SUM(Sales)) * 100, 2) AS Profit_Margin_Percentage
FROM Superstore
GROUP BY Discount
ORDER BY Discount;
""",conn)


,Discount,Number_of_Orders,Total_Sales,Total_Profit,Profit_Margin_Percentage
0,0.00,4798,1.087908e+06,320987.6032,29.51
1,0.10,94,5.436935e+04,9029.1770,16.61
2,0.15,52,2.755852e+04,1418.9915,5.15
3,0.20,3657,7.645944e+05,90337.3060,11.82
4,0.30,227,1.032267e+05,-10369.2774,-10.05
5,0.32,27,1.449346e+04,-2391.1377,-16.50
6,0.40,206,1.164178e+05,-23057.0504,-19.81
7,0.45,11,5.484974e+03,-2493.1111,-45.45
8,0.50,66,5.891854e+04,-20506.4281,-34.80
9,0.60,138,6.644700e+03,-5944.6552,-89.46


In [37]:
pd.read_sql_query("""
SELECT
    "Product Name",
    SUM(Sales) AS Total_Sales,
    SUM(Profit) AS Total_Profit,
    COUNT(*) AS Order_Count
FROM Superstore
GROUP BY "Product Name"
HAVING Total_Profit < 0
ORDER BY Total_Profit ASC;   -- most loss-making items first
""",conn)

,Product Name,Total_Sales,Total_Profit,Order_Count
0,Cubify CubeX 3D Printer Double Head Print,11099.963,-8.879970e+03,3
1,Lexmark MX611dhe Monochrome Laser Printer,16829.901,-4.589973e+03,4
2,Cubify CubeX 3D Printer Triple Head Print,7999.980,-3.839990e+03,1
3,Chromcraft Bull-Nose Wood Oval Conference Tabl...,9917.640,-2.876116e+03,5
4,Bush Advantage Collection Racetrack Conference...,9544.725,-1.934398e+03,7
...,...,...,...,...
296,"Brites Rubber Bands, 1 1/2 oz. Box",13.068,-5.148000e-01,3
297,Rubber Band Ball,58.344,-2.992000e-01,4
298,"Acco PRESSTEX Data Binder with Storage Hooks, ...",62.946,-1.614000e-01,4
299,Premier Electric Letter Opener,2641.608,-7.105427e-15,8


In [38]:
pd.read_sql_query("""
WITH seg AS (
    SELECT
        Segment,
        SUM(Sales) AS Segment_Sales
    FROM Superstore
    GROUP BY Segment
),
total AS (
    SELECT SUM(Sales) AS Total_Sales
    FROM Superstore
)
SELECT
    seg.Segment,
    seg.Segment_Sales,
    ROUND((seg.Segment_Sales * 100.0 / total.Total_Sales), 2) AS Contribution_Percentage
FROM seg, total
ORDER BY Contribution_Percentage DESC;
""",conn)


,Segment,Segment_Sales,Contribution_Percentage
0,Consumer,1.161401e+06,50.56
1,Corporate,7.061464e+05,30.74
2,Home Office,4.296531e+05,18.70


In [39]:
pd.read_sql_query("""
SELECT
    "Order ID",
    "Order Date",
    "Ship Date",
    julianday("Ship Date") - julianday("Order Date") AS Shipping_Time_Days
FROM Superstore
ORDER BY Shipping_Time_Days DESC;
""",conn)

,Order ID,Order Date,Ship Date,Shipping_Time_Days
0,CA-2016-152156,08-11-2016,11-11-2016,None
1,CA-2016-152156,08-11-2016,11-11-2016,None
2,CA-2016-138688,12-06-2016,16-06-2016,None
3,US-2015-108966,11-10-2015,18-10-2015,None
4,US-2015-108966,11-10-2015,18-10-2015,None
...,...,...,...,...
9989,CA-2014-110422,21-01-2014,23-01-2014,None
9990,CA-2017-121258,26-02-2017,03-03-2017,None
9991,CA-2017-121258,26-02-2017,03-03-2017,None
9992,CA-2017-121258,26-02-2017,03-03-2017,None


In [40]:
pd.read_sql_query("""
WITH stats AS (
    SELECT
        AVG(Sales) AS avg_sales,
        AVG((Sales - (SELECT AVG(Sales) FROM Superstore)) *
            (Sales - (SELECT AVG(Sales) FROM Superstore))) AS variance
    FROM Superstore
),
calc AS (
    SELECT
        avg_sales,
        sqrt(variance) AS std_dev
    FROM stats
)
SELECT
    S."Order ID",
    S."Product Name",
    S.Sales,
    S.Profit
FROM Superstore S, calc C
WHERE S.Sales > C.avg_sales + 2 * C.std_dev
ORDER BY S.Sales DESC;
""",conn)

,Order ID,Product Name,Sales,Profit
0,CA-2014-145317,Cisco TelePresence System EX90 Videoconferenci...,22638.480,-1811.0784
1,CA-2016-118689,Canon imageCLASS 2200 Advanced Copier,17499.950,8399.9760
2,CA-2017-140151,Canon imageCLASS 2200 Advanced Copier,13999.960,6719.9808
3,CA-2017-127180,Canon imageCLASS 2200 Advanced Copier,11199.968,3919.9888
4,CA-2017-166709,Canon imageCLASS 2200 Advanced Copier,10499.970,5039.9856
...,...,...,...,...
242,US-2017-134481,"Office Impressions End Table, 20-1/2""H x 24""W ...",1488.424,-297.6848
243,US-2015-163279,ClearOne CHATAttach 160 - speaker phone,1487.976,185.9970
244,CA-2014-166954,GuestStacker Chair with Chrome Finish Legs,1487.040,148.7040
245,CA-2014-125731,GuestStacker Chair with Chrome Finish Legs,1487.040,148.7040
